In [1]:
import pandas
import ee  # Google Earth Engine API
import folium
import numpy as np

In [2]:
ee.Initialize()

First define some utility functions.

In [3]:
def swap_lat_lng(coords: list):
    """Reverse the order of elements in each coordinate pair in a list"""
    coords = np.array([[c[1], c[0]] for c in coords])
    return coords
    
def to_number(key: str):
    """
    Return a lambda function that converts a feature's property value from a string to a number.
    To be used with map().
    
    param key: the name of the feature property to convert
    """
    return lambda feature: feature.set({key: ee.Number.parse(feature.get(key))})

## Mapping
Next we define a mapping class and some methods to help visualize the data. This will help with understanding what we're doing, and also make it easy to visually verify that the results make sense. The JavaScript version of the Earth Engine API provides a Map class that makes it easy to do this, but the Python API doesn't have that feature. I'm implementing it using folium as recommended here: https://developers.google.com/earth-engine/python_install-colab.html#interactive_map

These two methods are going to be added as custom folium map methods. This will make it easy to display vector and raster Earth Engine objects to a map.

In [4]:
def add_raster_layer(self, ee_image_object, vis_params, name):
    """Display an EE image (raster) on a folium map"""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data © Google Earth Engine",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

def add_vector_layer(self, geometry, name):
    """Display an EE geometry (vector) on a folium map"""
    coords = swap_lat_lng(np.array(geometry.coordinates().getInfo())[0])
    folium.vector_layers.Polygon(
        locations=coords,
        name=name,
        color='red',
        overlay=True,
        control=True
    ).add_to(self)

Next we define the Map class, which makes use of the above methods to draw multiple datasets onto a folium map and display it.

In [5]:
class Map:
    
    def __init__(self):
        """Initialize a custom folium map"""
        # Add EE drawing methods to folium.
        folium.Map.add_raster_layer = add_raster_layer
        folium.Map.add_vector_layer = add_vector_layer
        self.map = folium.Map(location=[39, -98], zoom_start=3, height=500)

    def add(self, datasets: list):
        """Add layers to an interactive map using folium"""
        
        # Add each dataset as a layer to the map
        for dataset in datasets:
            if isinstance(dataset, ee.Geometry):
                self.map.add_vector_layer(dataset, 'sample area')
                continue
            if isinstance(dataset, ee.FeatureCollection):
                dataset = dataset.reduceToImage(properties=['us_l3code'], reducer=ee.Reducer.first())
            
            # Set visualization parameters.
            vis_params = {
              'min': 0,
              'max': 120,
              'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}
            
            self.map.add_raster_layer(dataset.updateMask(dataset.gt(0)), vis_params, 'ecoregions')
            
    def display(self):
            self.map.add_child(folium.LayerControl())  # Add a layer control panel to the map
            display(self.map)   # Display the map


## Getting ecoregion data for a location
Here we find the name of the level III ecoregion in which the sample area is located. If the sample area contains multiple ecoregions, we choose the one that covers the largest amount of the sample area.

In [6]:

# Get the dataset from Earth Engine 
# and convert the level III ecoregion codes from strings to numbers
dataset = ee.FeatureCollection('EPA/Ecoregions/2013/L3')
dataset = dataset.map(to_number(key='us_l3code'))

# Example coordinates
lat = 34.083974
lng = -118.567312

# Define the sample area to be a ring around the sample location with a 100 meter radius
sample_area = ee.Geometry.Point(lng, lat).buffer(100)

# Filter the dataset to just the intersections of features with the sample area
# Usually there will be just one ecoregion feature covering the entire sample area,
# but it's possible that it could intersect multiple.
# In that case we will choose the one that covers the most of the sample area.
overlaps = dataset.filterBounds(sample_area).map(lambda feature: feature.intersection(sample_area))

# Add 'area' as a property to each feature
# This is the area in m^2 of the intersection of the ecoregion feature and the sample area
overlaps = overlaps.map(
    lambda feature: feature.set({'area': feature.geometry().area()}))
print("Number of ecoregions in the sample area:", overlaps.size().getInfo())

max_area = overlaps.aggregate_max('area')

# Get just the ecoregion with the largest 'area' property
predominant_ecoregion = overlaps.filter(
    ee.Filter.gte('area', max_area)).aggregate_array('us_l3name').getInfo()[0]
print("Predominant ecoregion in the sample area:", predominant_ecoregion)

# Display the map
map = Map()
map.add([dataset, overlaps, sample_area])
map.display()








Number of ecoregions in the sample area: 1
Predominant ecoregion in the sample area: Southern California/Northern Baja Coast
